### Notes 

T5 Paper: https://arxiv.org/pdf/1910.10683.pdf

T5 Tokenizer: https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_t5.py

Important Tasks: https://docs.google.com/document/d/1weIZM6QTlnitpPQmpg-WeV2RW70TnYmDuogBQPr5mB0/edit

In [1]:
#installation step
!pip install transformers
!pip install t5
!pip install sentencepiece
!pip install bpemb
#creating the folders 
!mkdir data/
!mkdir data/AD_NMT-master
!mkdir data/train/
!mkdir data/test/
!mkdir data/val/
#fetching the pkl files
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW' -O data/AD_NMT-master/english-Arabic-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V8_tp8ZlWUYaX7QQL46t0uSRNrVehSf1' -O data/AD_NMT-master/english-Arabic-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V7X0qtuDIyjTHY0wh-ZNoVwsiF4lId2e' -O data/AD_NMT-master/english-Arabic-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UzL4cOWTMCee83KBUh2QO_H62AFVpDQV' -O data/AD_NMT-master/LAV-MSA-2-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UpfCbkxhztof7dvNjeAs1bHjD4SER6h3' -O data/AD_NMT-master/LAV-MSA-2-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UlAZGtYsSfXzK7hrC_PbxQFqTSXD0DMw' -O data/AD_NMT-master/LAV-MSA-2-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UjDX7cCG2S23SPfSHxSPdVayMTxB5Y16' -O data/AD_NMT-master/Magribi_MSA-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UaVWIqRXo0rxuxDF4KArA4bEK1TaLX3l' -O data/AD_NMT-master/Magribi_MSA-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYvlhdYAdfa4riP_4hn3-IEVd1ZUXVTQ' -O data/AD_NMT-master/Magribi_MSA-train.pkl

     |████████████████████████████████| 645kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 9.7MB/s 
     |████████████████████████████████| 890kB 23.0MB/s 
     |████████████████████████████████| 3.8MB 22.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=5e3d20cfd2ea945cbb32df03e1625878b58fc23faae654b46caf645d82d9b2ed
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 143kB 2.8MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 3.3MB 9.1MB/s 
     |████████████████████████████████| 8.3MB 11.7MB/s 
     |████████████████████████████████| 296kB 42.1MB/s 
     |████████████████████████████████| 421.8MB 16kB/s 
     |████████████████████████████████| 450kB 40.5MB/s 
     |████████████████████████████████| 3.9MB 42.3MB/s 
  Created wheel for gast: filename=g

--2020-05-10 14:48:06--  https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW
Resolving docs.google.com (docs.google.com)... 173.194.215.101, 173.194.215.138, 173.194.215.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.215.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8u907f9s86l0h2nl2d20ia817sq2tsus/1589122050000/16970776037313924126/*/1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW?e=download [following]
--2020-05-10 14:48:07--  https://doc-10-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8u907f9s86l0h2nl2d20ia817sq2tsus/1589122050000/16970776037313924126/*/1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW?e=download
Resolving doc-10-2s-docs.googleusercontent.com (doc-10-2s-docs.googleusercontent.com)... 172.217.204.132, 2607:f8b0:400c:c15::84
Connecting to doc-10-2s-docs.googleusercontent.com (doc-10

In [0]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
from torch.optim import SGD
import pandas as pd
import numpy as np
import pickle
import os, io, glob
import functools

import sentencepiece as spm
from bpemb import BPEmb

import transformers
import t5
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

##Initial Loading from Pickle

In [3]:
ls data/AD_NMT-master

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    data_MSA_English_trainval = pickle.load(handle)

with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    data_MSA_English_test = pickle.load(handle)

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_MSA_English_both = pickle.load(handle) 




with open(file_path + "LAV-MSA-2-train.pkl", 'rb') as handle:
    data_LAV_MSA_trainval = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-test.pkl", 'rb') as handle:
    data_LAV_MSA_test = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA_both = pickle.load(handle) 




with open(file_path + "Magribi_MSA-train.pkl", 'rb') as handle:
    data_Magribi_MSA_trainval = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-test.pkl", 'rb') as handle:
    data_Magribi_MSA_test = pickle.load(handle) 

with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA_both = pickle.load(handle) 
    

In [5]:
#few dataset examples
print(data_MSA_English_both[0:5])
print(data_LAV_MSA_both[0:5])

#print length of train + val dataset, print length of both (train + val) + test
print(len(data_MSA_English_trainval))
print(len(data_MSA_English_both))

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش', 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة'], ['روح بوشك و فتول عاليسار', 'اذهب تقدم و استدر يسارا'], ['لا لا لازم انه يكون عندك موضوع ما في اشي', ' لا لا يجب ان يكون لديك موضوع هذا ضروري'], ['اوعي تبعدي من هون بلاش تضيعي ', 'لا تبتعد عن هنا حتى لا تفقد الطريق '], ['قصدي صراحة يما انا كمان كرهته من يوم ما عملتيه زي ما بتعمله خالتي كرهته و صرت ما باطيقه بالمرة', 'اقصد صراحة يا امي انا ايضا كرهته من يوم حضرته مثلما تحضره خالتي كرهته و اصبحت لا اطيقه ابدا']]
9000
10001


## Prepare Datasets

example: https://iwslt2010.fbk.eu/node/32/

We need to take our training and test sets from the pkl files and create new .txt files that are formatted so that the standard torchtext Dataset class can read them

Data format:
each line consists of three fields divided by the character '\'
sentences consisting of words divided by single spaces
format: <SENTENCE_ID>\<PARAPHRASE_ID>\<TEXT>
Field_1: sentence ID
Field_2: paraphrase ID
Field_3: MT develop sentence / reference translation
Text input example:
DEV_001\01\This is the first develop sentence.
DEV_002\01\This is the second develop sentence.
Reference translation example:
DEV_001\01\1st reference translation for 1st input
DEV_001\02\2nd reference translation for 1st input
...
DEV_002\01\1st reference translation for 2nd input
DEV_002\02\2nd reference translation for 2nd input
...
Languages:
Arabic-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
IWSLT07 testset: 489 sentences, 6 reference translations
IWSLT08 testset: 507 sentences, 16 reference translations
French-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
Turkish-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations


In [6]:
ls data/AD_NMT-master/

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
#splits the train dataset into train and validation sets, define test set as datafile
msa_en_train, msa_en_val = train_test_split(data_MSA_English_trainval, test_size=.2)
msa_en_test = data_MSA_English_test

lav_msa_train, lav_msa_val = train_test_split(data_LAV_MSA_trainval, test_size=.2)
lav_msa_test = data_LAV_MSA_test

mag_msa_train, mag_msa_val = train_test_split(data_Magribi_MSA_trainval, test_size=.2)
mag_msa_test = data_Magribi_MSA_test

In [8]:
print(len(msa_en_train))
print(len(msa_en_val))

print(len(lav_msa_train))
print(len(lav_msa_val))

print(len(mag_msa_train))
print(len(mag_msa_val))

7200
1800
11044
2761
14188
3548


In [0]:
file_path = 'data/'

def list_to_csv(ds, src='msa', trg='en', datatype=''):
    src_formatted = datatype + '_' + src + '_' + trg + '.' + 'csv'
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as csv:
        for i, arr in enumerate(ds):
            csv.write(arr[1] + ',' + arr[0] + '\n')

In [0]:
list_to_csv(msa_en_train, 'msa', 'en', 'train')
list_to_csv(msa_en_val, 'msa', 'en', 'val')
list_to_csv(msa_en_test, 'msa', 'en', 'test')

list_to_csv(lav_msa_train, 'lav', 'msa', 'train')
list_to_csv(lav_msa_val, 'lav', 'msa', 'val')
list_to_csv(lav_msa_test, 'lav', 'msa', 'test')

list_to_csv(mag_msa_train, 'mag', 'msa', 'train')
list_to_csv(mag_msa_val, 'mag', 'msa', 'val')
list_to_csv(mag_msa_test, 'mag', 'msa', 'test')

##Loading MultiBPemb: Multilingual BPEmb Sentencepiece Model

In [63]:
multibpemb = BPEmb(lang="multi", vs=10000, dim=300)

BPEmb fallback: multi from vocab size 10000 to 100000


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [64]:
print(multibpemb.EOS) #this might be an issue, as the SentencePieceVocabulary class in t5/data/sentencepiece_vocabulary.py assumes that ID=1 for EOS

2


In [65]:
filepath = str(multibpemb.model_file)
print(filepath)

/root/.cache/bpemb/multi/multi.wiki.bpe.vs100000.model


In [66]:
!git clone https://github.com/google/sentencepiece.git

fatal: destination path 'sentencepiece' already exists and is not an empty directory.


In [67]:
!apt-get install protobuf-compiler python-pil python-lxml

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-lxml is already the newest version (4.2.1-1ubuntu0.1).
python-pil is already the newest version (5.1.0-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [68]:
! mv sentencepiece/src/sentencepiece_model.proto ./sentencepiece_model.proto

mv: cannot stat 'sentencepiece/src/sentencepiece_model.proto': No such file or directory


In [69]:
! ls

data		  sample_data		      sentencepiece_model.proto
multibpemb.model  sentencepiece		      vcpkg
__pycache__	  sentencepiece_model_pb2.py


In [70]:
!protoc --python_out=. sentencepiece_model.proto
!ls *.py

sentencepiece_model_pb2.py


In [71]:
import sentencepiece_model_pb2 as model
m = model.ModelProto()
m.ParseFromString(open(filepath, 'rb').read())

1965223

In [0]:
m.pieces[0].piece = "<pad>"
m.pieces[0].type = 3
m.pieces[1].piece = "</s>"
m.pieces[1].type = 3
m.pieces[2].piece = "<unk>"
m.pieces[2].type = 2

In [73]:
print(m.pieces[0])
print(m.pieces[1])
print(m.pieces[2])

piece: "<pad>"
type: CONTROL

piece: "</s>"
type: CONTROL

piece: "<unk>"
type: UNKNOWN



In [0]:
with open('multibpemb.model', 'wb') as f:
  f.write(m.SerializeToString())

In [75]:
! ls

data		  sample_data		      sentencepiece_model.proto
multibpemb.model  sentencepiece		      vcpkg
__pycache__	  sentencepiece_model_pb2.py


In [0]:
filename = "multibpemb.model"

##Tensor Processing + Add to TaskRegistry

### MSA to English Task

In [0]:
msa_en_split_csv_path = {
    "train": "data/train/train_msa_en.csv",
    "validation": "data/val/val_msa_en.csv"
}
msa_en_example_count = {
    "train": 7200,
    "validation": 1800
}

In [78]:
def msa_en_translation_dataset_fn(split, shuffle_files=False):
  ds = tf.data.TextLineDataset(msa_en_split_csv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["",""],
                        field_delim=",", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE
  )
  ds = ds.map(lambda *example: dict(zip(["source", "target"], example)) )
  return ds

for example in tfds.as_numpy(msa_en_translation_dataset_fn("train").take(5)):
    print(example)

{'source': b'\xd8\xa8\xd8\xa7\xd9\x85\xd9\x83\xd8\xa7\xd9\x86\xd9\x87 \xd8\xa7\xd9\x84\xd9\x85\xd8\xac\xd9\x8a\xd8\xa1', 'target': b'He can come'}
{'source': b'\xd8\xaa\xd9\x88\xd9\x85 \xd9\x8a\xd9\x87\xd8\xb1\xd8\xa8', 'target': b'Tom is getting away'}
{'source': b'\xd8\xaa\xd9\x88\xd9\x85 \xd8\xb3\xd9\x8a\xd8\xb1\xd8\xba\xd8\xa8 \xd8\xa8\xd9\x87\xd8\xa7', 'target': b'Tom would like it'}
{'source': b'\xd8\xa7\xd8\xb9\xd9\x85\xd9\x84 \xd9\x83\xd9\x84 \xd9\x8a\xd9\x88\xd9\x85 \xd8\xa7\xd9\x84\xd8\xa7 \xd9\x8a\xd9\x88\xd9\x85 \xd8\xa7\xd9\x84\xd8\xa7\xd8\xad\xd8\xaf', 'target': b'I work every day except Sunday'}
{'source': b'\xd8\xb3\xd8\xa7\xd8\xad\xd8\xa7\xd9\x88\xd9\x84 \xd9\x85\xd8\xa7 \xd8\xa7\xd8\xb3\xd8\xaa\xd8\xb7\xd8\xb9\xd8\xaa \xd8\xa7\xd9\x84\xd8\xa7 \xd8\xa7\xd9\x82\xd8\xa7\xd8\xb7\xd8\xb9 \xd9\x85\xd8\xb0\xd8\xa7\xd9\x83\xd8\xb1\xd8\xaa\xd9\x83', 'target': b"I'll do my best not to disturb your studying"}


In [0]:
#turn the ds of dictionaries and change the keys to inputs and targets that the model
def msa_en_translation_preprocessor(ds):
  def to_inputs_and_targets(ex):
    return{
        "inputs": tf.strings.join(["Translate MSA to English: ",ex["source"]]),
        "targets": ex["target"]
    }
  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
t5.data.TaskRegistry.remove("msa_en_translation")
t5.data.TaskRegistry.add(
    #name of the Task
    "msa_en_translation",
    #Supply a function which returns a tf.data.Dataset
    dataset_fn=msa_en_translation_dataset_fn,
    splits=["train", "validation"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[msa_en_translation_preprocessor],
    # Use the same vocabulary that we used for pre-training.
    sentencepiece_model_path= filepath, #str(multibpemb.model_file), #t5.data.DEFAULT_SPM_PATH
    # Lowercase targets before computing metrics.
    postprocess_fn = t5.data.postprocessors.lower_text, 
    # We'll use accuracy as our evaluation metric.
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, but helps for mixing and auto-caching.
    num_input_examples=msa_en_example_count
)

###Levantine to MSA Task

In [0]:
lav_msa_split_csv_path = {
    "train": "data/train/train_lav_msa.csv",
    "validation": "data/val/val_lav_msa.csv"
}
lav_msa_example_count = {
    "train": 11044,
    "validation": 2761
}

In [82]:
def lav_msa_translation_dataset_fn(split, shuffle_files=False):
  ds = tf.data.TextLineDataset(lav_msa_split_csv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["",""],
                        field_delim=",", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE
  )
  ds = ds.map(lambda *example: dict(zip(["source", "target"], example)) )
  return ds

for example in tfds.as_numpy(lav_msa_translation_dataset_fn("train").take(5)):
    print(example)

{'source': b'\xd9\x83\xd8\xa7\xd9\x86\xd9\x83 \xd9\x86\xd9\x81\xd8\xae\xd8\xaa \xd8\xa8\xd8\xa7\xd9\x84\xd9\x88\xd9\x86\xd8\xa7 \xd8\xa7\xd8\xb0\xd8\xa7 \xd9\x86\xd8\xb2\xd8\xb9\xd8\xaa \xd9\x8a\xd8\xaf\xd9\x83 \xd8\xae\xd8\xb1\xd8\xac \xd8\xa7\xd9\x84\xd9\x87\xd9\x88\xd8\xa7\xd8\xa1', 'target': b'\xd9\x83\xd8\xa7\xd9\x86\xd9\x83 \xd9\x86\xd9\x81\xd8\xae\xd8\xaa \xd8\xa8\xd8\xa7\xd9\x84\xd9\x88\xd9\x86 \xd8\xa7\xd8\xb0\xd8\xa7 \xd8\xb4\xd9\x84\xd8\xaa \xd9\x84\xd9\x8a\xd8\xaf\xd9\x83 \xd8\xa8\xd9\x8a\xd8\xb7\xd9\x84\xd8\xb9 \xd8\xa7\xd9\x84\xd9\x87\xd9\x88\xd8\xa7'}
{'source': b' \xd8\xa7\xd9\x8a\xd9\x86 \xd8\xb0\xd9\x87\xd8\xa8\xd8\xaa   ', 'target': b'\xd9\x88\xd9\x8a\xd9\x86 \xd8\xb1\xd8\xad\xd8\xaa'}
{'source': b'\xd9\x85\xd8\xa7\xd8\xa8\xd9\x83  \xd9\x86\xd8\xad\xd9\x86 \xd9\x86\xd8\xb4\xd8\xa7\xd9\x87\xd8\xaf \xd9\x81\xd9\x84\xd9\x85\xd8\xa7 ', 'target': b'\xd8\xa7\xd9\x8a\xd8\xb4 \xd9\x85\xd8\xa7\xd9\x84\xd9\x83 \xd8\xa7\xd8\xad\xd9\x86\xd8\xa7 \xd8\xa8\xd9\x86\xd8\xaa\xd9\x81\x

In [0]:
#turn the ds of dictionaries and change the keys to inputs and targets that the model
def lav_msa_translation_preprocessor(ds):
  def to_inputs_and_targets(ex):
    return{
        "inputs": tf.strings.join(["Translate Levantine to MSA: ",ex["source"]]),
        "targets": ex["target"]
    }
  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
t5.data.TaskRegistry.remove("lav_msa_translation")
t5.data.TaskRegistry.add(
    #name of the Task
    "lav_msa_translation",
    #Supply a function which returns a tf.data.Dataset
    dataset_fn=lav_msa_translation_dataset_fn,
    splits=["train", "validation"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[lav_msa_translation_preprocessor],
    # Use the same vocabulary that we used for pre-training.
    sentencepiece_model_path= filepath, #str(multibpemb.model_file), #t5.data.DEFAULT_SPM_PATH
    # Lowercase targets before computing metrics.
    postprocess_fn = t5.data.postprocessors.lower_text, 
    # We'll use accuracy as our evaluation metric.
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, but helps for mixing and auto-caching.
    num_input_examples = lav_msa_example_count
)

###Maghrib to MSA Task

In [0]:
mag_msa_split_csv_path = {
    "train": "data/train/train_mag_msa.csv",
    "validation": "data/val/val_mag_msa.csv"
}
mag_msa_example_count = {
    "train": 14188,
    "validation": 3548
}

In [86]:
def mag_msa_translation_dataset_fn(split, shuffle_files=False):
  ds = tf.data.TextLineDataset(mag_msa_split_csv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["",""],
                        field_delim=",", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE
  )
  ds = ds.map(lambda *example: dict(zip(["source", "target"], example)) )
  return ds

for example in tfds.as_numpy(mag_msa_translation_dataset_fn("train").take(5)):
    print(example)
    print(example['source'].decode())
    print(example['target'].decode())
    print(len(example['source']))
    print(len(example['target']))

{'source': b' \xd9\x85\xd9\x86 \xd8\xa7\xd9\x84\xd9\x85\xd9\x81\xd8\xb1\xd9\x88\xd8\xb6 \xd8\xa7\xd9\x86\xd9\x83 \xd8\xaa\xd8\xb9\xd9\x84\xd9\x85 \xd9\x85\xd9\x86\xd8\xb0 \xd8\xa7\xd9\x84\xd8\xa8\xd8\xaf\xd8\xa7\xd9\x8a\xd8\xa9 \xd9\x84\xd8\xa7 \xd8\xaa\xd9\x82\xd9\x84 \xd8\xb4\xd9\x8a\xd8\xa6\xd8\xa7   ', 'target': b' \xd9\x86\xd9\x88\xd8\xb1\xd9\x85\xd8\xa7\xd9\x84\xd9\x85\xd9\x88\xd9\x86 \xd8\xb1\xd8\xa7\xd9\x83 \xd8\xb9\xd9\x84\xd9\x89 \xd8\xa8\xd8\xa7\xd9\x84\xd9\x83 \xd9\x85\xd9\x84\xd9\x88\xd9\x84 \xd9\x85\xd8\xa7 \xd8\xaa\xd9\x82\xd9\x88\xd9\x84\xd9\x8a\xd8\xb4 '}
 من المفروض انك تعلم منذ البداية لا تقل شيئا   
 نورمالمون راك على بالك ملول ما تقوليش 
82
70
{'source': b'\xd9\x86\xd8\xb9\xd9\x85', 'target': b'\xd8\xa7\xd9\x8a\xd9\x87'}
نعم
ايه
6
6
{'source': b'\xd8\xa8\xd9\x84 \xd8\xa7\xd9\x84\xd8\xa8\xd8\xb1\xd8\xaa\xd8\xba\xd8\xa7\xd9\x84\xd9\x8a', 'target': b'\xd9\x84\xd8\xa7\xd9\x84\xd8\xa7 \xd8\xa7\xd9\x84\xd8\xa8\xd8\xb1\xd8\xaa\xd8\xba\xd8\xa7\xd9\x84\xd9\x8a'}
بل البرتغال

In [0]:
#turn the ds of dictionaries and change the keys to inputs and targets that the model
def mag_msa_translation_preprocessor(ds):
  def to_inputs_and_targets(ex):
    return{
        "inputs": tf.strings.join(["Translate Maghrib to Arabic: ",ex["source"]]),
        "targets": ex["target"]
    }
  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
t5.data.TaskRegistry.remove("mag_msa_translation")
t5.data.TaskRegistry.add(
    #name of the Task
    "mag_msa_translation",
    #Supply a function which returns a tf.data.Dataset
    dataset_fn=mag_msa_translation_dataset_fn,
    splits=["train", "validation"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[mag_msa_translation_preprocessor],
    # Use the same vocabulary that we used for pre-training.
    sentencepiece_model_path=filepath, #t5.data.DEFAULT_SPM_PATH
    # Lowercase targets before computing metrics.
    postprocess_fn = t5.data.postprocessors.lower_text, 
    # We'll use accuracy as our evaluation metric.
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, but helps for mixing and auto-caching.
    num_input_examples = mag_msa_example_count
)

##Dataset Mixture

In [0]:
t5.data.MixtureRegistry.remove("ar_translation")
t5.data.MixtureRegistry.add(
    "ar_translation",
    ["msa_en_translation", "lav_msa_translation", "mag_msa_translation"],
     default_rate=1.0
)

##Fine Tune

In [90]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

#Using the Huggingface T5 Model

model = t5.models.HfPyTorchModel("t5-base", "/tmp/hft5/", device)

INFO:absl:Loading from /tmp/hft5/model-0.checkpoint


In [91]:
for i in range(15):
  print(m.pieces[i])

piece: "<pad>"
type: CONTROL

piece: "</s>"
type: CONTROL

piece: "<unk>"
type: UNKNOWN

piece: "00"
score: -0.0

piece: "an"
score: -1.0

piece: "\342\226\201d"
score: -2.0

piece: "en"
score: -3.0

piece: "er"
score: -4.0

piece: "\342\226\201s"
score: -5.0

piece: "in"
score: -6.0

piece: "\342\226\201p"
score: -7.0

piece: "ar"
score: -8.0

piece: "\342\226\201a"
score: -9.0

piece: "\342\226\20100"
score: -10.0

piece: "\342\226\201m"
score: -11.0



In [92]:
STEPS = 10000 #@param {type: "integer"}
model.train(
    mixture_or_task_name="ar_translation",
    steps=STEPS,
    save_steps=1000,
    sequence_length={"inputs": 128, "targets": 128},
    split="train",
    batch_size=32, #32
    optimizer=functools.partial(transformers.AdamW, lr=1e-4),
)

INFO:absl:Saving checkpoint for step 0


IndexError: ignored

##Evaluation

In [0]:
# Evaluate after fine-tuning
model.eval(
    "ar_translation",
    checkpoint_steps="all",
    sequence_length={"inputs": 64, "targets": 64},
    batch_size=128,
)

##Predictions

In [0]:
inputs = [
    "translation: This is a totally valid sentence.",
    "translation: A doggy detail was walking famously.",
]
model.predict(
    inputs,
    sequence_length={"inputs": 32},
    batch_size=2,
    output_file="/tmp/hft5/example_predictions.txt",
)